In [19]:
# 코스피 지수 구성 종목들의 링크 테이블 만들기.

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import ssl

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]

context = ssl._create_unverified_context()
html = urlopen('https://finance.naver.com/sise/sise_market_sum.naver?', context = context)
bs = BeautifulSoup(html, 'html.parser')
links = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})

table = []
for link in links:
        if 'href' in link.attrs:
            table.append(link.attrs['href'])
    
print("국내 코스피 50개 구성항목 링크")
            
for i in range(len(table)):
    table[i] = str(create_analysis_link(table[i]))

"""
우선 정보를 추출하여 저장하는 사이클 하나를 돈 이후에는,
이 부분을 함수로 만들어서 50개 이상의 항목들을 쭉 긁어올 수 있도록 만들어 놓자.
코드들의 부분 부분을 쭉 업그레이드를 해나갈 예정이다.
"""

print(table[13])

국내 코스피 50개 구성항목 링크
https://finance.naver.com/item/coinfo.naver?code=323410


In [20]:
# 얻어온 재무 분석 링크에서 javascript 링크 따오기.

array = []
for one in table:
    html = urlopen("{}".format(one), context = context)
    bs = BeautifulSoup(html,'html.parser')
    link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')
    if 'src' in link.attrs:
        goal = link.attrs['src']
        array.append(goal)
        
print(array[13])


https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=323410


In [21]:
# 따온 링크에서 원하는 정보 추출하기

import csv

def extract_name(name):
    a, b, c, d, name = name.split()
    i = -1
    for i in range(1, len(name)):
        if name[-i] == '(':
            name = name[-i+1:-1]
            break
    return name

csvFile = open('Finance.csv', 'wt+')
writer = csv.writer(csvFile)
try:

    writer.writerow(('기업 이름', 'EPS', 'BPS', 'PER', '업종PER', 'PBR', '현금배당수익률'))
    
    for one in array:
        html = urlopen("{}".format(one), context =context)
        bs = BeautifulSoup(html, 'html.parser')
        table = bs.findAll('table',{'class' : 'cmp-table'})[0]
        rows = table.findAll('tr')[2].findAll('dt')

        name = bs.find('title').get_text()
        name = extract_name(name)

        data = []
        for row in rows:
            if row.get_text() == '12월 결산':
                continue
            temp = row.get_text().split()
            if len(temp) == 1:
                temp.append('N/A')
            data.append(temp[1])
        
        writer.writerow((name, data[0],data[1],data[2],data[3],data[4],data[5]))
    
finally:
    csvFile.close()
    
print('the end...')

the end...


In [ ]:
"""
나름 소기의 목적을 달성했다.

여러 가지 과정을 통해 결국은 코스피 시총 1위부터 50위까지의 재무정보를 추출했고,
엑셀파일로도 깔끔하게 완성했다.

하지만, 페이지만 다른 51위 부터는 무엇때문인지 인덱스 오류가 생긴다.
아무래도 현금배당률이 제시가 안되어있다던지 등의 문제인 것 같은데, 
웹은 변수 투성이라는게 이런 말인가 싶다.

오히려 좋다. 에러처리를 직접 실습해볼 기회다!
"""